# Basic Transaction Network

This notebok demos building a simple transaction network. This will show the usage of: 
- The event-based scheduler
- Date-times as event stamps 
- Multiple types of agent
- Using network-x as a backend data-structure

In [1]:
import networkx as nx
from numpy.random import choice, randint, random
from datetime import datetime, timedelta

In [2]:
from functional_abm.schedulers.event_based import EventBasedScheduler
from functional_abm.agent import agent

## Network Generator

This function randomly generates a directed graph with nodes representing accounts and edges possible transaction paths. 

Each node is then randomly assigned a starting amount.

In [3]:
def generate_network(n_agents):
    net = nx.scale_free_graph(n_agents)
    
    starting_amounts = randint(0, 5000, n_agents)
    
    for i, n in enumerate(net.nodes):
        net.nodes[n]["id"] = str(i)
        net.nodes[n]["amount"] = starting_amounts[i]
        
    return net

## Model

The model takes the randomly generated graph and assignes the nodes to one of two types of agent:

- Savers: Only perform a transaction if the amount is greater 500 with 20% chance
- Spenders: Perform a transaction if they have any money with an 80% chance

The descendants of each agent (the nodes they can send money to) are the successors nodes on the directed transaction graph.

Running the model prints a series of transactions between nodes.

In [4]:
def transaction_network(n_agents: int, start_date: datetime, end_date: datetime):
    
    # Initialize transaction graph and scheduler
    # with model start and end times
    net = generate_network(n_agents)
    scheduler = EventBasedScheduler(end_date, start_time=start_date)
    
    # Create the `saver` agent type 
    @agent(scheduler=scheduler)
    def saver(t, antecedents, state, descendants):
        if descendants and state["amount"] > 500 and random()>0.8:
            send_to = choice(descendants)
            amount = randint(0,200)
            state["amount"] -= amount
            send_to["amount"] +=  amount
            print(f"Sending {amount:4} from {state['id']:3} to {send_to['id']:3} @ {t}")
        
        return t + timedelta(seconds=randint(60, 172800))
    
    
    # Create the `spender` agent type 
    @agent(scheduler=scheduler)
    def spender(t, antecedents, state, descendants):
        if descendants and state["amount"] > 0 and random()>0.2:
            send_to = choice(descendants)
            amount = randint(0,state["amount"])
            state["amount"] -= amount
            send_to["amount"] +=  amount
            print(f"Sending {amount:4} from {state['id']:3} to {send_to['id']:3} @ {t}")
        
        return t + timedelta(seconds=randint(60, 172800))
    
    
    # Assign the nodes to the agent types
    # along with the descendent taken from the directed graph
    for i, n in enumerate(net.nodes):
        start_time = start_date+timedelta(seconds=randint(0, 86400))
        if i%2 == 0:
            saver(start_time, [], 
                  net.nodes[n], 
                  [net.nodes[x] for x in net.successors(n)])
        else:
            spender(start_time, [], 
                    net.nodes[n], 
                    [net.nodes[x] for x in net.successors(n)])
    
    # Run the model
    while not scheduler.finished:
        scheduler.step()

In [5]:
foo = transaction_network(100, datetime(2020, 1, 1), datetime(2020, 1, 7))

Sending 2209 from 87  to 0   @ 2020-01-01 00:16:15
Sending  149 from 94  to 6   @ 2020-01-01 00:16:48
Sending  736 from 81  to 1   @ 2020-01-01 00:24:34
Sending   94 from 89  to 1   @ 2020-01-01 00:53:45
Sending  214 from 57  to 0   @ 2020-01-01 01:26:59
Sending 3313 from 31  to 0   @ 2020-01-01 02:26:33
Sending   27 from 14  to 8   @ 2020-01-01 02:48:56
Sending   43 from 13  to 1   @ 2020-01-01 03:42:33
Sending  134 from 91  to 0   @ 2020-01-01 04:25:12
Sending  412 from 23  to 6   @ 2020-01-01 04:35:33
Sending   87 from 47  to 1   @ 2020-01-01 04:38:53
Sending  147 from 8   to 99  @ 2020-01-01 04:54:08
Sending 2658 from 9   to 3   @ 2020-01-01 04:58:50
Sending   35 from 76  to 0   @ 2020-01-01 05:06:04
Sending 1697 from 67  to 1   @ 2020-01-01 05:26:15
Sending  107 from 18  to 16  @ 2020-01-01 05:46:01
Sending  178 from 4   to 23  @ 2020-01-01 06:08:51
Sending  191 from 25  to 0   @ 2020-01-01 07:01:39
Sending  128 from 89  to 1   @ 2020-01-01 07:23:15
Sending   62 from 46  to 18  @ 